## Stimulus coding with HDDMRegression

Note: This tutorial is more advanced. If you are just starting you might want
to head to the :ref:`demo <chap_demo>` instead.

In some situations it is useful to fix the magnitude of parameters
across stimulus types while also forcing them to have different
directions. For example, an independent variable could influence both
the drift rate ``v`` and the response bias ``z``. A specific example is an
experiment on face-house discrimination with different difficulty
levels, where the drift-rate is smaller when the task is more
difficult and where the bias to responding house is larger when the
task is more difficult.  One way to analyze the effect of difficulty
on drift rate and bias in such an experiment is to estimate one drift
rate ``v`` for each level, and a response bias ``z`` such that the bias for
houses-stimuli is ``z`` and the bias for face stimuli is ``1-z`` (``z = .5``
for unbiased decisions in ``HDDM``).

The following example describes how to generate simulated data for
such an experiment, how to set up the analysis with ``HDDMRegression``,
and compares true parameter values with those estimated with
``HDDMRegression``.

### Model Recovery Test for HDDMRegression

The test is performed with simulated data for an experiment with one
independent variable with three levels (e.g. three levels of
difficulty) which influence both drift rate ``v`` and bias ``z``. Responses
are "accuracy coded", i.e. correct responses are coded ``1`` and incorrect
responses ``0``. Further, stimulus coding of the parameter ``z`` is
implemented. "stimulus coding" of ``z`` means that we want to fit a model
in which the magnitude of the bias is the same for the two stimuli,
but its direction "depends on" the presented stimulus (e.g. faces or
house in a face-house discrimination task). Note that this does not
mean that we assume that decision makers adjust their bias after
having seen the stimulus. Rather, we want to measure response-bias (in
favor of face or house) while assuming the same drift rate for both
stimuli. We can achieve this for accuracy coded data by modeling the
bias as moved towards the correct response boundary for one stimulus
(e.g. ``z = .6`` for houses) and away from the correct response boundary
for the other stimulus (``1-z = .4`` for faces).

First, we need to import the required python modules.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from patsy import dmatrix  # for generation of (regression) design matrices
from pandas import Series  # to manipulate data-frames generated by hddm
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import hddm
import kabuki
print("The current version of kabuki is: ", kabuki.__version__)
print("The current version of HDDM is: ", hddm.__version__)

The current version of kabuki is:  0.6.5RC3
The current version of HDDM is:  0.8.0


We save the output of stdout to the file ``ModelRecoveryOutput.txt``.


In [2]:
import sys

sys.stdout = open("ModelRecoveryOutput.txt", "w")

#### Creating simulated data for the experiment

Next we set the number of subjects and the number of trials per level
for the simulated experiment

In [3]:
n_subjects = 10
trials_per_level = 150  # and per stimulus

Next we set up parameters of the drift diffusion process for the three
levels and the first stimulus. As desribed earlier ``v`` and ``z`` change
accross levels.

In [4]:
level1a = {"v": 0.3, "a": 2, "t": 0.3, "sv": 0, "z": 0.5, "sz": 0, "st": 0}
level2a = {"v": 0.4, "a": 2, "t": 0.3, "sv": 0, "z": 0.6, "sz": 0, "st": 0}
level3a = {"v": 0.5, "a": 2, "t": 0.3, "sv": 0, "z": 0.7, "sz": 0, "st": 0}

Now we generate the data for stimulus A

In [5]:
data_a, params_a = hddm.generate.gen_rand_data(
    {"level1": level1a, "level2": level2a, "level3": level3a},
    size=trials_per_level,
    subjs=n_subjects,
)

Next come the parameters for the second stimulus, where ``v`` is the same
as for the first stimulus. This is different for ``z``. In particular:
``z(stimulus_b) = 1 - z(stimulus_a)``. As a result, responses are
altogether biased towards responding A. Because we use accuracy coded
data, stimulus A is biased towards correct responses, and stimulus B
towards incorrect responses. 

In [6]:
level1b = {"v": 0.3, "a": 2, "t": 0.3, "sv": 0, "z": 0.5, "sz": 0, "st": 0}
level2b = {"v": 0.4, "a": 2, "t": 0.3, "sv": 0, "z": 0.4, "sz": 0, "st": 0}
level3b = {"v": 0.5, "a": 2, "t": 0.3, "sv": 0, "z": 0.3, "sz": 0, "st": 0}

Now we generate the data for stimulus B

In [15]:
data_b, params_b = hddm.generate.gen_rand_data(
    {"level1": level1b, "level2": level2b, "level3": level3b},
    size=trials_per_level,
    subjs=n_subjects,
)

We add a column to the ``DataFrame`` identifying stimulus A as 1 and stimulus B as 2.


In [16]:
data_a["stimulus"] = Series(np.ones((len(data_a))), index=data_a.index)
data_b["stimulus"] = Series(np.ones((len(data_b))) * 2, index=data_a.index)

Now we merge the data for stimulus A and B

In [17]:
mydata = pd.concat([data_a,data_b])
mydata.reset_index(drop=True,inplace=True)
mydata.head()

,rt,response,subj_idx,condition,stimulus
0,0.574638,0.0,0,level1,1.0
1,1.117408,1.0,0,level1,1.0
2,1.437870,0.0,0,level1,1.0
3,0.599739,0.0,0,level1,1.0
4,0.479797,1.0,0,level1,1.0


#### Setting up the HDDM regression model

Next we need to ensure that the bias is ``z`` for one stimulus and ``1-z``
for the other stimulus.  This is implemented here for all stimulus A trials
and -1 for stimulus B trials. We use the ``patsy`` command ``dmatrix`` to
generate such an array from the stimulus column of our simulated data


In [18]:
def z_link_func(x, data=mydata):
    stim = np.asarray(
        dmatrix("0 + C(s, [[0], [1]])", {"s": data.stimulus.loc[x.index]})
    )
    # Apply z = (1 - x) to flip them along 0.5
    z_flip = stim - x
    # The above inverts those values we do not want to flip,
    # so invert them back
    z_flip[stim == 0] *= -1
    return z_flip

(NOTE: earlier versions of this tutorial suggested applying an inverse logit
link function to the regression, but this should no longer be used given changes to the prior 
on the intercept.) 
Also depending on your python version, the above code may give you errors and you can try this instead:


In [19]:
def z_link_func(x, data=mydata):
    stim = np.asarray(
        dmatrix(
            "0 + C(s, [[0], [1]])",
            {"s": data.stimulus.loc[x.index]},
            return_type="dataframe",
        )
    )
    # Apply z = (1 - x) to flip them along 0.5
    z_flip = np.subtract(stim, x.to_frame())
    # The above inverts those values we do not want to flip,
    # so invert them back
    z_flip[stim == 0] *= -1
    return z_flip

Now we set up the regression models for ``z`` and ``v`` and also include the
link functions The relevant string here used by ``patsy`` is '1 +
C(condition)'. This will generate a design matrix with an intercept
(that's what the '1' is for) and two dummy variables for remaining
levels. (The column in which the levels are coded has the default name
'condition'):

In [20]:
z_reg = {"model": "z ~ 1 + C(condition)", "link_func": z_link_func}

For ``v`` the link function is simply ``x = x``, because no transformations is
needed. [However, you could also analyze this experiment with response
coded data. Then you would not stimulus code ``z`` but ``v`` and you would
have to multiply the ``v`` for one condition with ``-1``, with a link function
like the one for ``z`` above, but with out the additional logit transform
]

In [21]:
v_reg = {"model": "v ~ 1 + C(condition)", "link_func": lambda x: x}

Now we can finally put the regression description for the hddm model
together. The general template for this is ``[{'model': 'outcome_parameter ~ patsy_design_string', 'link_func': your_link_function }, {...}, ...]``


In [22]:
reg_descr = [z_reg, v_reg]

The last step before running the model is to construct the complete hddm regression model by adding data etc.

In [23]:
m_reg = hddm.HDDMRegressor(mydata, reg_descr, include=["v", "a", "t", "z"])

Now we start the model, and wait for a while (you can go and get
several coffees, or read a paper). 

In [24]:
m_reg.sample(2000, burn=100)

#### Comparing generative and recovered model parameters

First we print the model stats

In [29]:
m_reg.gen_stats()

,mean,std,2.5q,25q,50q,75q,97.5q,mc err
a,1.998861,0.040545,1.917624,1.974326,1.997989,2.024473,2.080622,0.001152
a_std,0.120709,0.038479,0.068639,0.094533,0.112498,0.137538,0.218302,0.001343
a_subj.0,1.986982,0.03271,1.922713,1.965166,1.988089,2.009385,2.052463,0.001026
a_subj.1,1.812338,0.031551,1.752925,1.790734,1.812222,1.832901,1.876891,0.001124
a_subj.2,1.901954,0.037589,1.83622,1.876826,1.898992,1.924445,1.986716,0.001722
a_subj.3,2.049442,0.038348,1.975284,2.023818,2.049156,2.073306,2.131397,0.001197
a_subj.4,2.137115,0.035922,2.068358,2.112721,2.136027,2.161699,2.20794,0.001308
a_subj.5,1.934417,0.032355,1.872632,1.912715,1.934565,1.956781,2.000017,0.000991
a_subj.6,2.060654,0.037044,1.990156,2.036267,2.059152,2.084699,2.137146,0.001211
a_subj.7,1.966539,0.032227,1.902371,1.944619,1.966208,1.988026,2.03199,0.000824


Lets first look at ``v``. For ``level1`` this is just the
intercept. The value of ``.347`` is in the ball park of the true value
of ``.3``. The fit is not perfect, but running a longer chain might
help (we are ignoring sophisticated checks of model convergence for
this example here). To get the values of ``v`` for levels 2 and 3, we
have to add the respective parameters (``0.122`` and ``.263``) to the
intercept value. The resulting values of  are again
close enough to the true values of ``.4`` and ``.5``. The ``z_Intercept``
value of .504 is close tothe true value of ``.5``, and the level 2 and level 3
offsets are also close (.504 + .096 = .600 and .504 +.195 = 0.699).   In sum,
``HDDMRegression`` easily recovered the right order of the parameters
``z``. The recovered parameter values are also close to the true
parameter values, and this was only for a single subject fit.
Parameter estimates are improved with more subjects. 